In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
import os
import numpy as np
import cv2
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from math import ceil

In [ ]:
from chessutils import find_coeffs

In [ ]:
from boardgen import moirebackground
from boardgen import chessboard

# Datagenerator

In [ ]:
PATH_TO_IMG = 'img'

In [ ]:
MAXSHEAR = 0.15
MINSCALE = 0.5

In [ ]:
figimgs = [f for f in os.listdir(PATH_TO_IMG) if f.split('_')[0]=='Chess']
figuresimgs = dict()
for f in figimgs:
    fn = f.split('_')[1].split('4')[0]
    img = cv2.imread(os.path.join(PATH_TO_IMG, f))
    figuresimgs[fn] = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

figs = ['p', 'b', 'n', 'r', 'q', 'k']
colors = ['d', 'l']

In [ ]:
parameters = {
    'numcell':8,
    'cellsize':45,
    'figures':figs,
    'colors':colors,
    'shear':MAXSHEAR,
    'scale':MINSCALE,
}

In [ ]:
IMGSIZE = 480

In [ ]:
SIZE = IMGSIZE // parameters['numcell']

In [ ]:
figdict = {f:i+1 for i,f in enumerate(figs)}
coldict = {c:i+1 for i,c in enumerate(colors)}

In [ ]:
def boarddistortgen(imgsize):
    boardimage, recs, vecs = chessboard(figuresimgs, np.random.rand(13), imgsize, parameters)

    background = moirebackground(np.random.rand(8), imgsize)
    result = (background * np.asarray(boardimage)).astype(np.uint8)
    distortedvec = np.clip(vecs + 10 * (np.random.rand(8) - 0.5), 0, imgsize-1)
    
    coeffs = find_coeffs(
        [(0, 0), (imgsize, 0), (imgsize, imgsize), (0, imgsize)],
        np.reshape(distortedvec, (4,2)))
    
    img = Image.fromarray(result, 'L')

    img = img.transform((imgsize,imgsize), Image.PERSPECTIVE, coeffs, Image.BICUBIC)
    stacked = np.asarray(img)
   
    return stacked, recs

In [ ]:
def separate(stacked, records):
    boardsize = stacked.shape[0]
    numcell = parameters['numcell']
    size = boardsize // numcell
    fieldnum = numcell*numcell
    res = np.zeros((fieldnum,size,size))
    cls = np.zeros((fieldnum,2))
    for i, f, c in records:
        cls[i][0] = figdict[f]
        cls[i][1] = coldict[c]
    for i in range(fieldnum):
        x = i % numcell
        y = i // numcell
        res[i] = stacked[y*size:(y+1)*size,x*size:(x+1)*size]
    return np.expand_dims(res, axis=-1), cls

In [ ]:
bd, rc = boarddistortgen(IMGSIZE)

In [ ]:
bsd, rsd = separate(bd, rc)

In [ ]:
plt.imshow(bd, cmap='gray')
plt.show()

In [ ]:
plt.imshow(bd[0], cmap='gray')
plt.title(f"{int(rc[0][0])}:{int(rc[0][1])}")
plt.show()

In [ ]:
bd.shape

In [ ]:
def boarddistortgen(imgsize):
    boardimage, recs, vecs = chessboard(figuresimgs, np.random.rand(13), imgsize, parameters)

    background = moirebackground(np.random.rand(8), imgsize)
    result = (background * np.asarray(boardimage)).astype(np.uint8)
    distortedvec = np.clip(vecs + 10 * (np.random.rand(8) - 0.5), 0, imgsize-1)
    
    coeffs = find_coeffs(
        [(0, 0), (imgsize, 0), (imgsize, imgsize), (0, imgsize)],
        np.reshape(distortedvec, (4,2)))
    
    img = Image.fromarray(result, 'L')
    img = img.transform((imgsize,imgsize), Image.PERSPECTIVE, coeffs, Image.BICUBIC)
    
    stacked = np.asarray(img)
    
    numcell = parameters['numcell']
    size = imgsize // numcell
    fieldnum = numcell*numcell
    res = np.zeros((fieldnum,size,size))
    cls = np.zeros((fieldnum,2))
    for i, f, c in recs:
        cls[i][0] = figdict[f]
        cls[i][1] = coldict[c]
    for i in range(fieldnum):
        x = i % numcell
        y = i // numcell
        res[i] = stacked[y*size:(y+1)*size,x*size:(x+1)*size]
    return np.expand_dims(res, axis=-1), cls

In [ ]:
def boardgen():
    bd, rc = boarddistortgen(IMGSIZE)
    count = 0
    while True:
        if (count == bd.shape[0]):
            bd, rc = boarddistortgen(IMGSIZE)
            count = 0
        yield bd[count], rc[count]
        count += 1

In [ ]:
dataset = tf.data.Dataset.from_generator(boardgen,
                                         output_signature=
                                         (tf.TensorSpec(shape=(SIZE,SIZE,1), dtype=tf.float32),
                                          tf.TensorSpec(shape=(2), dtype=tf.float32))
                                        )

In [ ]:
for t in dataset.take(5):
    print(t[0].shape, t[1].shape)

In [ ]:
trainset = dataset.batch(64)
valset = dataset.batch(12)

In [ ]:
for t in trainset.take(2):
    print(t[0].shape, t[1].shape)

In [ ]:
for t in valset.take(2):
    print(np.squeeze(t[1].numpy().astype(int)).T)

# CLASSIFIER

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dense, MaxPooling2D, Concatenate

In [ ]:
def classnet():

    inputs = tf.keras.layers.Input(shape=(SIZE,SIZE,1))

    x = inputs
    fsize = [16, 32, 64, 128]

    for i in range(4):
        x = Conv2D(filters = fsize[i],
                kernel_size = (3, 3),
                kernel_initializer = 'he_normal',
                padding = 'same', strides=(2,2), activation='relu', name=f'cnv1_{i}')(x)

    x = GlobalAveragePooling2D(name='glob')(x)
    output_fig = Dense(7, activation = 'softmax', name='fig')(x)
    output_col = Dense(3, activation = 'softmax', name='col')(x)
    output = Concatenate(name='unite')([output_fig, output_col])
    
    model = Model(inputs=inputs, outputs=output, name='class_board')

    return model

In [ ]:
md = classnet()

In [ ]:
md.summary()

In [ ]:
def custom_loss(y_actual, y_pred):

    scc = tf.keras.losses.SparseCategoricalCrossentropy()
    part1 = scc(y_actual[:,0:1],y_pred[:,0:7])
    part2 = scc(y_actual[:,1:2],y_pred[:,7:10])

    return part1 + part2

In [ ]:
class colAccuracy(tf.keras.metrics.SparseCategoricalAccuracy):

    def __init__(self, name='col_acc', **kwargs):
        super(colAccuracy, self).__init__(name=name, **kwargs)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_actual = y_true[:,1:2]
        y_predicted = y_pred[:,7:10]
        return super(colAccuracy,self).update_state(y_actual,y_predicted,sample_weight)

In [ ]:
class figAccuracy(tf.keras.metrics.SparseCategoricalAccuracy):

    def __init__(self, name='fig_acc', **kwargs):
        super(figAccuracy, self).__init__(name=name, **kwargs)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_actual = y_true[:,0:1]
        y_predicted = y_pred[:,0:7]
        return super(figAccuracy,self).update_state(y_actual,y_predicted,sample_weight)

In [ ]:
md.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=custom_loss,
               metrics=[figAccuracy(), colAccuracy()])

In [ ]:
md.evaluate(valset, steps=128)

In [ ]:
EPOCHS = 1
model_history = md.fit(
    trainset,
    steps_per_epoch=500,
    epochs=EPOCHS,
    validation_data=valset,
    validation_steps=10)

In [ ]:
md.save('models/class_figure_col_v1.h5')

In [ ]:
##################################################################

In [ ]:
md = tf.keras.models.load_model('models/class_figure_col_v2.h5', compile=False)